<h1>EDA

In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

# file_path = "./Data/"
# df_imdb_title_basics = pd.read_csv(file_path + 'imdb.title.basics.csv.gz')
# df_tn_movie_budgets = pd.read_csv(file_path + 'tn.movie_budgets.csv.gz')

In [3]:
#Define the filepaths for all stored data in the Data directory in order to read into dfs
zipped = !ls ./Data/
path = "./Data/"
zipped_files = [path+x for x in zipped]
zipped_files

['./Data/bom.movie_gross.csv.gz',
 './Data/imdb.name.basics.csv.gz',
 './Data/imdb.title.akas.csv.gz',
 './Data/imdb.title.basics.csv.gz',
 './Data/imdb.title.crew.csv.gz',
 './Data/imdb.title.principals.csv.gz',
 './Data/imdb.title.ratings.csv.gz',
 './Data/rt.movie_info.tsv.gz',
 './Data/rt.reviews.tsv.gz',
 './Data/tmdb.movies.csv.gz',
 './Data/tn.movie_budgets.csv.gz']

In [4]:
#Function to read in named dict of dfs
def read_all(namelist):
    """Read contents of zippedData into a named dictionary with names of
    files as keys, dataframes as values.
    
    Takes a list of full filepaths.
    """
    name_dict = {}
    for filename in namelist:
        #extract the portion of the filename corresponding to the "name"
        name = filename[7:-7]
        if filename.endswith('csv.gz'):
            tmp_df = pd.read_csv(filename)
            name_dict[name] = tmp_df
        elif filename.endswith('tsv.gz'):
        #both tsv are encoded in ascii per cmd line and chardet, but for some reason 'latin' appears to do the trick:
            tmp_df = pd.read_csv(filename, sep='\t', encoding='latin')
            name_dict[name] = tmp_df
        #in case there are any additional files in the dir
        else:
            continue
    return name_dict

In [5]:
#Make the dictionary of dataframes and name the keys
df_dict = read_all(zipped_files)
print(df_dict.keys())

dict_keys(['bom.movie_gross', 'imdb.name.basics', 'imdb.title.akas', 'imdb.title.basics', 'imdb.title.crew', 'imdb.title.principals', 'imdb.title.ratings', 'rt.movie_info', 'rt.reviews', 'tmdb.movies', 'tn.movie_budgets'])


In [6]:
#Will be working primarily with the tn_movie_budgets and imdb_title_basics data
df_imdb_title_basics = df_dict['imdb.title.basics']
df_tn_movie_budgets = df_dict['tn.movie_budgets']

In [7]:
# split the genres column up into a list. For easier joining, do not explode until after joined,
# retain original genre column to show the combinations if useful later

df_imdb_title_basics['genre_list'] = df_imdb_title_basics['genres'].str.split(",")
print(df_imdb_title_basics.dtypes)
df_imdb_title_basics.head()


tconst              object
primary_title       object
original_title      object
start_year           int64
runtime_minutes    float64
genres              object
genre_list          object
dtype: object


,tconst,primary_title,original_title,start_year,runtime_minutes,genres,genre_list
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama","[Action, Crime, Drama]"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama","[Biography, Drama]"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,[Drama]
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama","[Comedy, Drama]"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy","[Comedy, Drama, Fantasy]"


In [8]:
df_tn_movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [9]:
# Cleaning the tn_movie_budgets dataframe

# Modify the budget and gross columns to be numeric
df_tn_movie_budgets['production_budget'] = df_tn_movie_budgets['production_budget'] \
                                            .map(lambda x: x.strip('$').replace(',','')).astype(float)
df_tn_movie_budgets['domestic_gross'] = df_tn_movie_budgets['domestic_gross'] \
                                            .map(lambda x: x.strip('$').replace(',','')).astype(float)
df_tn_movie_budgets['worldwide_gross'] = df_tn_movie_budgets['worldwide_gross'] \
                                            .map(lambda x: x.strip('$').replace(',','')).astype(float)

# Create a Year column to help join with other dfs in case of movie title duplicates
df_tn_movie_budgets['year'] = df_tn_movie_budgets['release_date'] \
                                .map(lambda x: int(x[-4:]))

# Define Domestic and Worldwide profit columns as gross - production_budget
df_tn_movie_budgets['domestic_profit'] = df_tn_movie_budgets['domestic_gross'] \
                                        - df_tn_movie_budgets['production_budget']
df_tn_movie_budgets['worldwide_profit'] = df_tn_movie_budgets['worldwide_gross'] \
                                        - df_tn_movie_budgets['production_budget']
df_tn_movie_budgets['domestic_roi'] = df_tn_movie_budgets['domestic_gross'] \
                                        / df_tn_movie_budgets['production_budget']
df_tn_movie_budgets['worldwide_roi'] = df_tn_movie_budgets['worldwide_gross'] \
                                        / df_tn_movie_budgets['production_budget']

df_tn_movie_budgets.head()



,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,year,domestic_profit,worldwide_profit,domestic_roi,worldwide_roi
0,1,"Dec 18, 2009",Avatar,425000000.0,760507625.0,2.776345e+09,2009,335507625.0,2.351345e+09,1.789430,6.532577
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09,2011,-169536125.0,6.350639e+08,0.587101,2.546673
2,3,"Jun 7, 2019",Dark Phoenix,350000000.0,42762350.0,1.497624e+08,2019,-307237650.0,-2.002376e+08,0.122178,0.427892
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09,2015,128405868.0,1.072414e+09,1.388403,4.243841
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000.0,620181382.0,1.316722e+09,2017,303181382.0,9.997217e+08,1.956408,4.153696


In [10]:
budget_by_genre = df_tn_movie_budgets.merge(df_imdb_title_basics, left_on=['movie', 'year'], right_on=['primary_title', 'start_year'])
budget_by_genre.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 1547 entries, 0 to 1546
Data columns (total 18 columns):
id                   1547 non-null int64
release_date         1547 non-null object
movie                1547 non-null object
production_budget    1547 non-null float64
domestic_gross       1547 non-null float64
worldwide_gross      1547 non-null float64
year                 1547 non-null int64
domestic_profit      1547 non-null float64
worldwide_profit     1547 non-null float64
domestic_roi         1547 non-null float64
worldwide_roi        1547 non-null float64
tconst               1547 non-null object
primary_title        1547 non-null object
original_title       1547 non-null object
start_year           1547 non-null int64
runtime_minutes      1521 non-null float64
genres               1541 non-null object
genre_list           1541 non-null object
dtypes: float64(8), int64(3), object(7)
memory usage: 229.6+ KB


In [11]:
# This adds the studio to the dataset
df_bom_movie_gross = df_dict['bom.movie_gross']
df_bom_movie_gross.drop(columns = ['domestic_gross', 'foreign_gross'], inplace = True)
df_budget_genre_studio = budget_by_genre \
    .merge(df_bom_movie_gross, left_on=['movie', 'year'], right_on=['title', 'year'])

df_budget_genre_studio.tail()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,year,domestic_profit,worldwide_profit,domestic_roi,worldwide_roi,tconst,primary_title,original_title,start_year,runtime_minutes,genres,genre_list,title,studio
1047,84,"Aug 2, 2013",The Canyons,250000.0,59671.0,62375.0,2013,-190329.0,-187625.0,0.238684,0.24950,tt2292959,The Canyons,The Canyons,2013,99.0,"Drama,Thriller","[Drama, Thriller]",The Canyons,IFC
1048,48,"Jul 22, 2011",Another Earth,175000.0,1321194.0,2102779.0,2011,1146194.0,1927779.0,7.549680,12.01588,tt1549572,Another Earth,Another Earth,2011,92.0,"Drama,Romance,Sci-Fi","[Drama, Romance, Sci-Fi]",Another Earth,FoxS
1049,80,"Jul 10, 2015",The Gallows,100000.0,22764410.0,41656474.0,2015,22664410.0,41556474.0,227.644100,416.56474,tt2309260,The Gallows,The Gallows,2015,81.0,"Horror,Mystery,Thriller","[Horror, Mystery, Thriller]",The Gallows,WB (NL)
1050,86,"Jul 7, 2017",A Ghost Story,100000.0,1594798.0,2769782.0,2017,1494798.0,2669782.0,15.947980,27.69782,tt6265828,A Ghost Story,A Ghost Story,2017,92.0,"Drama,Fantasy,Romance","[Drama, Fantasy, Romance]",A Ghost Story,A24
1051,18,"Nov 12, 2010",Tiny Furniture,50000.0,391674.0,424149.0,2010,341674.0,374149.0,7.833480,8.48298,tt1570989,Tiny Furniture,Tiny Furniture,2010,98.0,"Comedy,Drama,Romance","[Comedy, Drama, Romance]",Tiny Furniture,IFC


In [12]:
# This separates the directors and writers attributes into lists
df_imdb_title_crew = df_dict['imdb.title.crew']
df_imdb_title_crew['directors_list'] = df_imdb_title_crew['directors'].str.split(",")
df_imdb_title_crew['writers_list'] = df_imdb_title_crew['writers'].str.split(",")
df_imdb_title_crew.tail()

,tconst,directors,writers,directors_list,writers_list
146139,tt8999974,nm10122357,nm10122357,[nm10122357],[nm10122357]
146140,tt9001390,nm6711477,nm6711477,[nm6711477],[nm6711477]
146141,tt9001494,"nm10123242,nm10123248",NaN,"[nm10123242, nm10123248]",NaN
146142,tt9004986,nm4993825,nm4993825,[nm4993825],[nm4993825]
146143,tt9010172,NaN,nm8352242,NaN,[nm8352242]


In [29]:
df_names = df_dict['imdb.name.basics']

,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
606643,nm9990381,Susan Grobes,NaN,NaN,actress,NaN
606644,nm9990690,Joo Yeon So,NaN,NaN,actress,"tt9090932,tt8737130"
606645,nm9991320,Madeline Smith,NaN,NaN,actress,"tt8734436,tt9615610"
606646,nm9991786,Michelle Modigliani,NaN,NaN,producer,NaN
606647,nm9993380,Pegasus Envoyé,NaN,NaN,"director,actor,writer",tt8743182


In [10]:
# Contains budget and gross sales with the genre list attribute by title/tconst and release year
budget_by_genre.to_csv('./budget_by_genre.csv')

# Modified the budget_by_genre dataset to include studio
df_budget_genre_studio.to_csv('./budget_genre_studio.csv')

# Contains the nconst director and writer attribute lists by tconst
df_imdb_title_crew.to_csv('./crew.csv')

# Contains the name attribute by nconst
df_names.to_csv('./names.csv')